In [4]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [5]:
filepath = '/home/otto/data/atmos-flux-data/processed/ATMOS_L3_2023-04-07_fluxgapfill.csv'

df = (pd.read_csv(filepath, low_memory=False).assign(TIMESTAMP_END = lambda x: x['DATE'], 
                                                     FCH4 = lambda x: x['ch4_flux'], 
                                                     sin_DOY = lambda x: np.sin(2*np.pi*x['DOY']/365), 
                                                     cos_DOY = lambda x: np.cos(2*np.pi*x['DOY']/365))

     )

In [7]:
fp = f'/home/otto/data/atmos-flux-data/methane-gapfill/data_dir/ATMOS_20230407_baseline/raw.csv'
df.to_csv(fp, index = False)
# df.replace(np.nan, -9999)

In [8]:
df

,DATE,filename,date,time,DOY,daytime,file_records,used_records,Tau,qc_Tau,...,hour_local.met,min_local.met,time_local.met,DOY_local.met,obs1,VPD_hPa,TIMESTAMP_END,FCH4,sin_DOY,cos_DOY
0,2022-05-24 10:30:00,2022-05-24T100000_AIU-2189.ghg,2022-05-24,10:30,144.4374,1,18000,18000,-9999.000000,0,...,10,30,10:30,144.42,1,9.545268,2022-05-24 10:30:00,-0.009860,0.609332,-0.792915
1,2022-05-24 11:00:00,2022-05-24T103000_AIU-2189.ghg,2022-05-24,11:00,144.4582,1,18000,18000,-0.124060,0,...,11,0,11: 0,144.46,2,9.675918,2022-05-24 11:00:00,-0.035292,0.609048,-0.793134
2,2022-05-24 11:30:00,not_enough_data,2022-05-24,11:30,144.4790,-9999,-9999,-9999,-9999.000000,-9999,...,11,30,11:30,144.46,3,-9999.000000,2022-05-24 11:30:00,-9999.000000,0.608764,-0.793352
3,2022-05-24 12:00:00,not_enough_data,2022-05-24,12:00,144.4990,-9999,-9999,-9999,-9999.000000,-9999,...,12,0,12: 0,144.50,4,-9999.000000,2022-05-24 12:00:00,-9999.000000,0.608491,-0.793561
4,2022-05-24 12:30:00,2022-05-24T120000_AIU-2189.ghg,2022-05-24,12:30,144.5207,1,18000,18000,-0.191470,1,...,12,30,12:30,144.50,5,12.662257,2022-05-24 12:30:00,-0.107285,0.608194,-0.793788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12764,2023-02-14 08:30:00,2023-02-14T080000_AIU-2189.ghg,2023-02-14,08:30,45.3541,1,18000,18000,-0.115257,0,...,8,30,8:30,45.34,12765,5.325785,2023-02-14 08:30:00,-9999.000000,0.703802,0.710397
12765,2023-02-14 09:00:00,2023-02-14T083000_AIU-2189.ghg,2023-02-14,09:00,45.3749,1,18000,18000,-0.192124,0,...,9,0,9: 0,45.38,12766,7.055078,2023-02-14 09:00:00,-0.001679,0.704056,0.710145
12766,2023-02-14 09:30:00,2023-02-14T090000_AIU-2189.ghg,2023-02-14,09:30,45.3957,1,18000,18000,-0.302091,0,...,9,30,9:30,45.38,12767,9.038119,2023-02-14 09:30:00,-0.001289,0.704310,0.709892
12767,2023-02-14 10:00:00,2023-02-14T093000_AIU-2189.ghg,2023-02-14,10:00,45.4166,1,18000,18000,-0.362121,0,...,10,0,10: 0,45.42,12768,8.960148,2023-02-14 10:00:00,-0.005689,0.704566,0.709639


In [21]:
df.columns[df.columns.str.contains('Star')]

Index(['NEE_uStar_f', 'NEE_uStar_fqc', 'NEE_uStar_fall', 'NEE_uStar_fall_qc',
       'NEE_uStar_fnum', 'NEE_uStar_fsd', 'NEE_uStar_fmeth', 'NEE_uStar_fwin'],
      dtype='object')

In [ ]:
from fluxgapfill import *

sites = 'ATMOS_20230407_all'

'''
1) for testing, input "TEST"
2) for one site, input the name of the site data folder, e.g., "MPM"
3) for multiple sites, input a series of sites devided by ",", e.g., "MPM, rf"
''' 

models=['rf','ann']

'''
1) input a list of model names devided by ",", e.g., ['lasso', 'ann']
models supported: 'lasso', 'ann', 'rf'

'''
# fp=f'{data_dir}{sites}/raw.csv'
# df = pd.read_csv(fp, nrows=1)
# df[df.columns[df.columns.str.contains('u.')]]

# predictors = 'DOY,sin_DOY,cos_DOY,H,LE,co2_flux,h2o_flux,air_temperature,air_pressure,RH,VPD,wind_speed,wind_dir,Tsoil_mean,SWC_mean,SW_IN_POT' # 2023-04-05
# predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure' # baseline
# predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,Tsoil_mean,SWC_mean' # base + soil
# predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,NEE_f,Reco,GPP_f,LE,H' # base + fluxes
# predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir' # base + met
# predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir,Tsoil_mean,SWC_mean' # base + met + soil
predictors = 'DOY,sin_DOY,cos_DOY,air_temperature,SW_IN_POT,wind_speed,air_pressure,u.,VPD,Precip,RH,wind_dir,Tsoil_mean,SWC_mean,NEE_f,Reco,GPP_f,LE,H' # all: base + met + soil + fluxes
'''
1) input a list of predictors. Comma-separated list of predictors. Ignored if
                                predictors_path is provided.
                                Certain keyword predictors are used to denote
                                specific sets of predictors:

2) input shortcuts. 
"all": all usable predictors in the data
"temporal": 

3) input a list of paths to files having predictor names. Comma-separated list path file(s) with
                                      predictor names. See predictors/metereological.txt for an
                                      example.
'''

In [ ]:
for predictor_subset in predictors:    

In [ ]:
preprocess(sites=sites, 
           na_values=-9999, # set a different na_values if needed
           data_dir=data_dir)
train(sites=sites,
      data_dir=data_dir,
      models=models,
      predictors=predictors,
      overwrite_existing_models=True)

test(data_dir=data_dir,
     sites=sites,
     models=models,
     predictors='all',
     split='test',
     distribution='laplace',
     overwrite_results=True
     )

fp=f'{data_dir}{sites}/gap.csv'
gap_df = pd.read_csv(fp)
gap_df = gap_df.fillna(-9999)
gap_df.to_csv(fp, index = False)

gapfill(data_dir=data_dir, 
        sites=sites,
        models=models)